<a href="https://colab.research.google.com/github/rybread1/trump-speech-writer/blob/master/trump_speech_writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
import itertools
from collections import Counter
import re
import os

In [2]:
pip install transformers

     |████████████████████████████████| 1.0MB 8.6MB/s 
     |████████████████████████████████| 3.0MB 25.4MB/s 
     |████████████████████████████████| 890kB 53.6MB/s 
     |████████████████████████████████| 1.1MB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f831fc5b5d195812527d1a69e918bb0834d75de043c1f767a387a2ccbe9894a2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!git clone https://github.com/rybread1/trump_speech_writer

Cloning into 'trump_speech_writer'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 68 (delta 25), reused 34 (delta 9), pack-reused 0
Unpacking objects: 100% (68/68), done.


In [4]:
## Reading and processing text
with open('/content/trump_speech_writer/speeches.txt', 'r') as fp:
    text = fp.read()

text = text[6:]

In [5]:
import nltk
nltk.download('punkt')
text_split = nltk.tokenize.sent_tokenize(text)

text_split_ = []

for ex in text_split:
  text_split_.append('<|startoftext|>' + ex + '<|endoftext|>')

text = ' '.join(text_split_)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [127]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>')
gpt_model = TFGPT2LMHeadModel.from_pretrained('gpt2')

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [128]:
tokenized_text = tokenizer.encode(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (280002 > 1024). Running this sequence through the model will result in indexing errors


In [129]:
examples = []
block_size = 100
for i in range(0, len(tokenized_text)-block_size+1, block_size): # Truncate in block of block_size
    examples.append(tokenized_text[i:i+block_size])

inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])

In [130]:
input = tf.keras.layers.Input(shape=(block_size-1,), dtype=tf.int32)
output = gpt_model(input)[0]
model = tf.keras.Model(inputs=input, outputs=output)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * gpt_model.config.n_layer], metrics=[metric])
model.fit(dataset, epochs=3)

Epoch 1/3


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


 20/218 [=>............................] - ETA: 1:18 - loss: nan - accuracy: 0.0133

KeyboardInterrupt: ignored

In [6]:
from transformers import TextDataset
from sklearn.model_selection import train_test_split

def build_text_files(split_texts, dest_path):

    f = open(dest_path, 'w')
    data = ''

    for texts in split_texts:

        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "

    f.write(data)

train, test = train_test_split(text_split, test_size=0.15)
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [7]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>')

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [8]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):

    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=32)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=32)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,

    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [9]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2_model", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler

    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,

)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:256: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


In [10]:
trainer.train()

{'loss': 3.3924111328125, 'learning_rate': 5e-05, 'epoch': 1.0121457489878543, 'total_flos': 190924513099776, 'step': 500}


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 2.95086376953125, 'learning_rate': 4.4369369369369375e-05, 'epoch': 2.0242914979757085, 'total_flos': 381849026199552, 'step': 1000}



{'loss': 2.70072021484375, 'learning_rate': 3.873873873873874e-05, 'epoch': 3.0364372469635628, 'total_flos': 572773539299328, 'step': 1500}



{'loss': 2.5095693359375, 'learning_rate': 3.310810810810811e-05, 'epoch': 4.048582995951417, 'total_flos': 763698052399104, 'step': 2000}



...

{'loss': 2.2322666015625, 'learning_rate': 2.1846846846846845e-05, 'epoch': 6.0728744939271255, 'total_flos': 1145547078598656, 'step': 3000}



{'loss': 2.13253125, 'learning_rate': 1.6216216216216218e-05, 'epoch': 7.08502024291498, 'total_flos': 1336471591698432, 'step': 3500}



{'loss': 2.059515625, 'learning_rate': 1.0585585585585586e-05, 'epoch': 8.097165991902834, 'total_flos': 1527396104798208, 'step': 4000}



{'loss': 2.003001953125, 'learning_rate': 4.954954954954955e-06, 'epoch': 9.109311740890687, 'total_flos': 1718320617897984, 'step': 4500}




TrainOutput(global_step=4940, training_loss=2.4362632844129553)

In [11]:
trainer.save_model()

In [12]:
from transformers import pipeline

chef = pipeline('text-generation', model='./gpt2_model', tokenizer='gpt2', config={'max_length':1000})

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [16]:
result = chef('iving generations of joblessness and poverty to millions of African American and Hispanic workers at a fraction of the price of tuition at the private public public and private colleges and universities.')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [17]:
result[0]['generated_text']

"iving generations of joblessness and poverty to millions of African American and Hispanic workers at a fraction of the price of tuition at the private public public and private colleges and universities.  And that's the one thing I learned.  We have too much at"

In [24]:
initial_seed = 'Wow! What a beautiful crowd here tonight!'
speech = initial_seed

for i in range(50):
  result = chef(initial_seed)
  speech += result[0]['generated_text'][len(initial_seed):]
  initial_seed = result[0]['generated_text'][len(initial_seed):]



Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad

In [25]:
speech

'Wow! What a beautiful crowd here tonight!  We also have the endorsement of Sheriff Joe.  Now the whole country hears my message but if Donald Trump gets that nomination, they may as well get - they may as well have to leave the convention, because they have got to vote.  Now, just because we have an immigration system that goes through, or because of something or somebody, doesn\'t mean it has to be corrected, because they\'ve got to use it as soon as possible.  She has also been clear about her support for regime change in Syria, and the destabilization of Lebanon, which she has supported violently with the use of chemical weapons.  We want to have the most inclusive school choice in the land.  That\'s why I\'ve done a lot of work with Hispanic youth.  We have the biggest tax cut since Ronald Reagan and the biggest trade cut since Bill Clinton\'s NAFTA.  You\'ve lost one of the great states, you\'ve lost two of the great states.  Our jobs have been taken away and we\'re putting them 

In [ ]:
# tokenize full text into individual words
rex_tokenizer = nltk.tokenize.TreebankWordTokenizer()
text_tokens = rex_tokenizer.tokenize(text)

word_set = set(text_tokens)
word_set_sorted = sorted(word_set)

word_2_int_dict = {ch:i for i,ch in enumerate(word_set_sorted)}
word_array = np.array(word_set_sorted)

text_encoded = np.array(
    [word_2_int_dict[ch] for ch in text_tokens],
    dtype=np.int32)

In [ ]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

seq_length = 20 
ds_chunks = ds_text_encoded.batch(seq_length+1, drop_remainder=True) 

## define the function for splitting x & y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 20000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

def get_test_train_split(text, seq_length, batch_size, train_split=0.8):
    return round(np.floor(len(text) / seq_length / batch_size) * train_split)

train_batches = get_test_train_split(text_encoded, seq_length, BATCH_SIZE, train_split=0.7)
print('train batches: ', train_batches)

ds_train = ds.take(train_batches)
ds_valid = ds.skip(train_batches)

train batches:  139.0


In [ ]:
def build_model(input_size, vocab_size, embedding_dim, rnn_units, dropout=True):
    inputs = tf.keras.Input(input_size)
    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    x = tf.keras.layers.LSTM(rnn_units, return_sequences=True)(x)
    x = tf.keras.layers.Dense(rnn_units)(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

tf.random.set_seed(1)

model = build_model(input_size=seq_length, vocab_size=len(word_array), 
                    embedding_dim=100, rnn_units=128)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()
checkpoint_path = "training_1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                       patience=3, 
                                                       restore_best_weights=True)


results = model.fit(ds_train, 
                    validation_data=ds_valid, 
                    epochs=100, 
                    callbacks=[early_stop_callback, 
                               cp_callback])



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 100)           1236500   
_________________________________________________________________
lstm (LSTM)                  (None, 20, 128)           117248    
_________________________________________________________________
dense (Dense)                (None, 20, 128)           16512     
_________________________________________________________________
dense_1 (Dense)              (None, 20, 12365)         1595085   
Total params: 2,965,345
Trainable params: 2,965,345
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
139/139 [==============================] - 8s 58ms/step - loss: 7.2925 - accuracy: 0.0496 - val_lo

In [ ]:
def generate_text(model, starting_str, 
           len_generated_text=500, 
           max_input_length=80,
           scale_factor=1.0):
    
    starting_str = starting_str.lower()
    encoded_input = [word_2_int_dict[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)        
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(scaled_logits, num_samples=1)
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()          
        generated_str += str(word_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)

        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

def generate_text_words(model, starting_str, len_generated, max_input_length, scale_factor):
    tokened_starting_string = rex_tokenizer.tokenize(starting_str)
    encoded_input = [word_2_int_dict[s] for s in tokened_starting_string]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str
    model.reset_states()
    for i in range(len_generated):
        logits = model.predict(encoded_input)
        logits = tf.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        new_word_indx = tf.random.categorical(scaled_logits, num_samples=1)
        new_word_indx = tf.squeeze(new_word_indx)[-1].numpy()
        new_word_indx = tf.expand_dims([new_word_indx], 0)
        generated_str += ' ' + str(word_array[new_word_indx][0])
        encoded_input = tf.concat(
                    [encoded_input, new_word_indx],
                    axis=1)

        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

In [ ]:
generated_text = generate_text_words(model, 
                               starting_str="We have to start winning again. We haven't wont in a long time.", 
                               scale_factor=4, 
                               len_generated=300,
                               max_input_length=seq_length)

print(generated_text)

KeyError: ignored

In [ ]:
from nltk.tokenize import sent_tokenize

sent_tokenize(generated_text)